Escolhi a fonte de dados abertos da CVM contendo a cotação diária dos fundos de investimentos negociados no mercado brasileiro. Essa fonte é atualizada diariamente com os dados de fechamento do dia anterior e as contações são agrupadas por arquivos correspondentes a cada mês do ano.

Esse é um cenário bem comum em fontes de dados abertas, uma série de arquivos no formato CSV agrupapando informações de acordo com a data, então  a solução que vamos implementar durante o tutorial é reaproveitável para outras fontes de dados.

Primeiro é importante analisar a fonte de dados, entender como ela está estruturada, quais campos compõem o dataset e como nós podemos automatizar a coleta dos dados.

A fonte de dados contendo a cotação diária dos fundos pode ser acessada através do [portal de dados abertos](http://www.dados.gov.br) pelo link:

[http://www.dados.gov.br/dataset/fi-doc-inf_diario](http://www.dados.gov.br/dataset/fi-doc-inf_diario)

Aqui tem uma descrição rápida sobre as informações que existem no dataset:
![](img/descricao_dataset.png)

Além de um link para o recurso contendo o dicionário de dados que descreve cada campo do dataset, é **sempre muito importante analisar o dicionário de dados do dataset, se disponível.** Não vou entrar em detalhes sobre as descrições do dicionário para não estender o tutorial, mas sugiro que você olhe para ser familiarizar com esse tipo de formato.

![](img/link_dicionario.png)

Agora partindo para como nós podemos automatizar a extração dos datasets, como podemos observar no site existe um link para todos os datasets dos últimos 12 meses. Primeiro precisamos analisar se existe um padrão nas urls de download dos arquivos, para isso vamos copiar alguns endereços e compará-los.

Para copiar os endereços clique com o botão direito no botão **"Ir para recurso"** e selecione a opção **"Copiar link"**:

![](img/link_recurso.png)

Copiei três links e vamos compará-los a seguir:

[http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_201907.csv](http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_201907.csv)

[http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_201910.csv](http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_201910.csv)

[http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202003.csv](http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202003.csv)

Como podemos observar existe um padrão claro nos links:`dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_YYYYMM.csv`. A única variação é o mês e o ano de referência do dataset. Com essa análise nossa tarefa de automatizar o download dos datasets ficou mais simples, porque agora podemos gerar sistematicamente o intervalo de datas que queremos baixar.

In [1]:
from datetime import datetime, timedelta

Primeiramente vamos criar uma função para facilitar a criação da lista de datas que nós queremos baixar do site

In [12]:
def generate_dates(initial_date, final_date, increment, format):
    date_list = []
    date_list.append(initial_date.strftime(format))
    
    while initial_date < final_date:
        initial_date = initial_date + timedelta(days=increment)
        date_list.append(initial_date.strftime(format))
        
    return date_list